In [7]:
import geopandas as gpd
import pandas as pd
import rasterio as rio
import rasterio.plot
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import shutil
import subprocess
import tempfile
import xarray as xr
# import stackstac
import rasterio.mask
from rasterio.features import geometry_mask
import warnings
import configparser
warnings.filterwarnings('ignore')

In [8]:
def run_command(cmd,shell_bool=False):
    """Safely runs a command, and returns the returncode silently in case of no error. Otherwise,
    raises an Exception
    """
    res = subprocess.run(cmd, check=True, capture_output=True,shell=shell_bool)
    
    if res.returncode != 0:
        log.error(f"Error with return code {res.returncode}")
        raise Exception
    return res.returncode

In [9]:
# def _get_cmd_precip_download(outputpath,link,version,secrets):
   
#     cmd = [
#         "wget",
#         "-O",
#         outputpath,
#         "--user",
#         f'{secrets["imerg"]["username"]}',
#         '--password',
#         f'{secrets["imerg"]["pwd"]}',
#         link,
#         '--no-proxy'
#     ]
#     return cmd

In [10]:
# Obtaining IMERG Final precipitation data

def _get_cmd_precip_download(outputpath,link,version,secrets):
    # Define the command (different for FINAL, same for EARLY and LATE)
    if version == "IMERG-FINAL":
        cmd = [
            "curl",
            '-o',
            outputpath,
            '--ssl-reqd',
            '-u',
            f'{secrets["imerg"]["username"]}:{secrets["imerg"]["pwd"]}',
            link
        ]
       
    else:
        cmd = [
            "wget",
            "-O",
            outputpath,
            "--user",
            f'{secrets["imerg"]["username"]}',
            '--password',
            f'{secrets["imerg"]["pwd"]}',
            link,
            '--no-proxy'
        ]
    return cmd


In [11]:
secrets = configparser.ConfigParser()
secrets.read('../../secrets/secrets.ini')

['../../secrets/secrets.ini']

In [12]:
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2021-12-31')



for date in pd.date_range(start_date, end_date):
    print(f"Downloading IMERG-FINAL precipitation data for: {date.strftime('%Y-%m-%d')}")
    curr_date = date
    base_date = pd.datetime(int(date.year),1,1)
    curr_date = pd.datetime(int(date.year),int(date.month),int(date.day))
    date_delta = curr_date - base_date
    imerg_final_id = date_delta.days*30
    if imerg_final_id <1000:
        if imerg_final_id <30:
            imerg_final_id = f'000{imerg_final_id}'
        elif imerg_final_id <100:
            imerg_final_id = f'00{imerg_final_id}'
        else:
            imerg_final_id = f'0{imerg_final_id}'

    version = "IMERG-FINAL"
    # version = "IMERG-LATE"
    outputpath = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL/{date.strftime('%Y-%m-%d')}_IMERG.tif"
    outputpath_zip = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_ZIP/{date.strftime('%Y-%m-%d')}_IMERG.zip"
    outputpath_temp = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_TEMP/{date.strftime('%Y-%m-%d')}_IMERG"
    if not os.path.isdir(outputpath_temp):
        os.makedirs(outputpath_temp)
    
    with tempfile.TemporaryDirectory(dir=outputpath_temp) as outputPathTemp:
       
        # link = f"https://arthurhouhttps.pps.eosdis.nasa.gov/gpmdata/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%d')}/gis/3B-DAY-GIS.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S000000-E235959.0000.V06B.tif"
        # link = f"https://jsimpsonhttps.pps.eosdis.nasa.gov/imerg/gis/{date.strftime('%Y')}/{date.strftime('%m')}/3B-HHR-L.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S233000-E235959.1410.V06B.1day.tif"
        link = f"https://arthurhouhttps.pps.eosdis.nasa.gov/gpmdata/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%d')}/gis/3B-DAY-GIS.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S000000-E235959.{imerg_final_id}.V06B.zip"


        cmd = _get_cmd_precip_download(outputpath,link,version,secrets)
        if not(os.path.exists(outputpath_zip)):
            response = requests.head(link,auth=(secrets["imerg"]["username"], secrets["imerg"]["pwd"]))
            print(response)
            if response.status_code == 200 :       
                cmd = _get_cmd_precip_download(outputpath_zip,link,version,secrets)
                run_command(cmd, shell_bool=False)
        
        #Unizipping IMERG_FINAL zip file to temp folder and extracting total accumulated precipitation file
        cmd = ["unzip",
                   f'{outputpath_zip}',
                   "-d",
                   f'{outputPathTemp}'
                ]
        run_command(cmd)
            
        if not(os.path.exists(outputpath)):
            path_totalPrecip = os.path.join(outputPathTemp,f"3B-DAY-GIS.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S000000-E235959.{imerg_final_id}.V06B.total.accum.tif")
            cmd = ["mv",
                       f'{path_totalPrecip}',
                       f'{outputpath}'
                    ]
            run_command(cmd)
    
    

AttributeError: module 'pandas' has no attribute 'datetime'

In [344]:
link

'https://arthurhouhttps.pps.eosdis.nasa.gov/gpmdata/2018/07/01/gis/3B-DAY-GIS.MS.MRG.3IMERG.20180701-S000000-E235959.5430.V06B.zip'

In [29]:
def process_precip(basin_bounds,srcpath, dstpath, temp_datadir=None):
    """For any IMERG Precipitation file located at `srcpath` is clipped, scaled and converted to
    ASCII grid file and saved at `dstpath`. All of this is done in a temporarily created directory
    which can be controlled by the `datadir` path
    """
    if temp_datadir is not None and not os.path.isdir(temp_datadir):
        raise Exception(f"ERROR: {temp_datadir} directory doesn't exist")
    
    if not(os.path.exists(dstpath)):
        # log.debug("Processing Precipitation file: %s", srcpath)
        
        print('**')
        with tempfile.TemporaryDirectory(dir=temp_datadir) as tempdir:
            clipped_temp_file = os.path.join(tempdir, 'clipped.tif')
            
            # unzip /home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_ZIP/2018-07-01_IMERG.zip 
            # -d /home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_temp/2018-07-01_IMERG
            
            # mv /home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_temp/2018-07-01_IMERG/3B-DAY-GIS.MS.MRG.3IMERG.20180701-S000000-E235959.5430.V06B.liquid.accum.tif 
            # /home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_RAW/2018-07-01_IMERG.tif
            
            #Unzipping IMERG Final file and extracting only accumulation precip file. Then moving it into IMERG_FINAL_RAW folder
                                
            cmd = [
                "gdalwarp",
                "-dstnodata", 
                "-9999.0",
                "-tr",
                "0.0625",
                "0.0625",
                "-te",
                str(basin_bounds[0]),
                str(basin_bounds[1]),
                str(basin_bounds[2]),
                str(basin_bounds[3]),
                '-of',
                'GTiff',
                '-overwrite', 
                f'{srcpath}',
                clipped_temp_file
            ]
            # print(cmd)
            run_command(cmd)

            # Scale down (EARLY)
            
            scaled_temp_file = os.path.join(tempdir, 'scaled.tif')
            cmd = [
                "gdal_calc.py", 
                "-A", 
                clipped_temp_file, 
                f"--calc=A*0.1", 
                f"--outfile={scaled_temp_file}", 
                "--NoDataValue=-9999", 
                "--format=GTiff"
            ]
            run_command(cmd)

            # Change format, and save as processed file
            aai_temp_file = os.path.join(tempdir, 'processed.tif')
            cmd = [
                'gdal_translate',
                '-of', 
                'aaigrid', 
                scaled_temp_file, 
                aai_temp_file
                ]
            run_command(cmd)

            # Move to destination
            print('***')
            shutil.move(aai_temp_file, dstpath)
    else:
        print(f"Processing Precipitation file exist: {dstpath}")

In [30]:
basin_df = gpd.read_file('keralaBasins_dissolved.geojson')
basin_bounds = basin_df.total_bounds

start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2018-12-31')
for date in pd.date_range(start_date, end_date):
    print(f"processing IMERG_FINAL precipitation data for : {date.strftime('%Y-%m-%d')}")
    srcpath = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL/{date.strftime('%Y-%m-%d')}_IMERG.tif"
    dstpath = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_processed/{date.strftime('%Y-%m-%d')}_IMERG.asc"
    temp_datadir = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_temp/{date.strftime('%Y-%m-%d')}_IMERG"
    if not os.path.isdir(temp_datadir):
        os.makedirs(temp_datadir)
    process_precip(basin_bounds,srcpath,dstpath,temp_datadir)

processing IMERG_FINAL precipitation data for : 2017-01-01
**
***
processing IMERG_FINAL precipitation data for : 2017-01-02
**
***
processing IMERG_FINAL precipitation data for : 2017-01-03
**
***
processing IMERG_FINAL precipitation data for : 2017-01-04
**
***
processing IMERG_FINAL precipitation data for : 2017-01-05
**
***
processing IMERG_FINAL precipitation data for : 2017-01-06
**
***
processing IMERG_FINAL precipitation data for : 2017-01-07
**
***
processing IMERG_FINAL precipitation data for : 2017-01-08
**
***
processing IMERG_FINAL precipitation data for : 2017-01-09
**
***
processing IMERG_FINAL precipitation data for : 2017-01-10
**
***
processing IMERG_FINAL precipitation data for : 2017-01-11
**
***
processing IMERG_FINAL precipitation data for : 2017-01-12
**
***
processing IMERG_FINAL precipitation data for : 2017-01-13
**
***
processing IMERG_FINAL precipitation data for : 2017-01-14
**
***
processing IMERG_FINAL precipitation data for : 2017-01-15
**
***
processing

In [47]:
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2018-12-31')

kerala_precip_IMERGLATE = []
kerala_precip_IMERGFINAL = []
for date in pd.date_range(start_date, end_date):
    precip_fn_FINAL = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_FINAL_processed/{date.strftime('%Y-%m-%d')}_IMERG.asc"
    precip_fn = f"/home/jovyan/Project/Kerala_2018_Floods/IMERG_LATE/precipitation/{date.strftime('%Y-%m-%d')}_IMERG.asc"
    ds = xr.open_rasterio(precip_fn)
    ds_FINAL = xr.open_rasterio(precip_fn_FINAL)
    states = gpd.read_file('/home/jovyan/Project/Kerala_2018_Floods/kerala.geojson')
    geoms = states.geometry.values
    mask = geometry_mask(geoms, out_shape=ds.isel(band=0).shape, transform=ds.transform)
    clipped_ds = ds.where(~mask)
    clipped_ds_FINAL = ds_FINAL.where(~mask)
    print(date, f"Average precip - LATE: {clipped_ds.mean().values}mm, Average precip - FINAL: {clipped_ds_FINAL.mean().values}mm")
    kerala_precip_IMERGLATE.append({'precipitation': clipped_ds.mean().values,
                          'time': pd.to_datetime(date)})
    kerala_precip_IMERGFINAL.append({'precipitation': clipped_ds_FINAL.mean().values,
                          'time': pd.to_datetime(date)})

2017-01-01 00:00:00 Average precip - LATE: 0.0mm, Average precip - FINAL: 0.0mm
2017-01-02 00:00:00 Average precip - LATE: 0.0mm, Average precip - FINAL: 0.0012254901960784314mm
2017-01-03 00:00:00 Average precip - LATE: 0.0mm, Average precip - FINAL: 0.0mm
2017-01-04 00:00:00 Average precip - LATE: 0.0mm, Average precip - FINAL: 0.0012254901960784314mm
2017-01-05 00:00:00 Average precip - LATE: 0.006127450980392157mm, Average precip - FINAL: 0.007352941176470588mm
2017-01-06 00:00:00 Average precip - LATE: 0.003676470588235294mm, Average precip - FINAL: 0.011029411764705883mm
2017-01-07 00:00:00 Average precip - LATE: 0.01715686274509804mm, Average precip - FINAL: 0.020833333333333332mm
2017-01-08 00:00:00 Average precip - LATE: 0.011029411764705883mm, Average precip - FINAL: 0.012254901960784314mm
2017-01-09 00:00:00 Average precip - LATE: 0.00857843137254902mm, Average precip - FINAL: 0.023284313725490197mm
2017-01-10 00:00:00 Average precip - LATE: 0.0024509803921568627mm, Average 

In [48]:
kerala_precip_IMERGLATE = pd.DataFrame(kerala_precip_IMERGLATE)
kerala_precip_IMERGLATE.set_index('time', inplace=True)
kerala_precip_IMERGFINAL = pd.DataFrame(kerala_precip_IMERGFINAL)
kerala_precip_IMERGFINAL.set_index('time', inplace=True)

In [49]:
kerala_rainfall_insitu = pd.read_csv('kerala_precip_201718.csv')
kerala_rainfall_insitu['time'] = pd.to_datetime(kerala_rainfall_insitu['Date'])
kerala_rainfall_insitu['date'] = kerala_rainfall_insitu['time'].dt.date
kerala_rainfall_insitu.set_index('time', inplace=True)

In [50]:
kerala_rainfall_201718 = kerala_rainfall_insitu.copy()
kerala_rainfall_201718['IMERG_LATE'] = kerala_precip_IMERGLATE['precipitation']
kerala_rainfall_201718['IMERG_FINAL'] = kerala_precip_IMERGFINAL['precipitation']
kerala_rainfall_201718.to_csv('kerala_rainfall_validation_201718.csv')